# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to save bottom fields of omega aragonite, omega calcite and pH as netcdf files
# requires full model output
# note that some paths are hard-coded throughout the script

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd
from tqdm import tqdm

In [2]:
#------
# MODEL INFO
#------
    
# load mesh
path_mesh = '/pscratch/sd/c/cnissen/'
file_mesh = 'Nissen2022_FESOM_REcoM_mesh_information_corrected_20220910.nc'

f1 = Dataset(path_mesh+file_mesh) #xr.open_dataset(path+file1)
lat      = f1.variables['lat'][:]
lon      = f1.variables['lon'][:]
zlevs    = f1.variables['zlevs'][:]
cavities = f1.variables['cavity'][:]
topo     = f1.variables['topo'][:]
area     = f1.variables['cell_area'][:]
volume   = f1.variables['cell_volume'][:]
f1.close()
print(lat.shape)
ind_cavities = np.where(cavities==1)[0]


(72411,)


In [20]:
#-----
# load model data & extract bottom layer omega A, omega C, pH 
#-----

which_sim = 'simAssp585'

path = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/'+which_sim+'/' 

vari_list = ['omegaA_bottom','omegaC_bottom','pH_bottom']

#year_list1 = np.arange(2090,2099+1,1) 
year_list1 = np.arange(1990,1999+1,1) 

#if which_sim in ['simAssp126','simAssp245','simAssp370','simCssp245']:
#    year_list1 = np.arange(2015,2100+1,1) 
#elif which_sim in ['simAssp585','simB','simCssp585']:
#    year_list1 = np.arange(1990,2100+1,1) 

omegaA_bottom_model1 = np.zeros([len(year_list1),len(lon)])
omegaC_bottom_model1 = np.zeros([len(year_list1),len(lon)])
pH_bottom_model1     = np.zeros([len(year_list1),len(lon)])
for yy in range(0,len(year_list1)):
    print('Process year '+str(year_list1[yy])+' of '+which_sim)
    
    path  = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/'+which_sim+'/'
    f1    = Dataset(path+'OmegaA_fesom_'+str(year_list1[yy])+'0101.nc') 
    f2    = Dataset(path+'OmegaC_fesom_'+str(year_list1[yy])+'0101.nc') 
    f3    = Dataset(path+'pH_fesom_'+str(year_list1[yy])+'0101.nc')
    data_mean1 = f1.variables['OmegaA'][:,:,:].mean(axis=0)  # depth x mesh.n2d
    data_mean2 = f2.variables['OmegaC'][:,:,:].mean(axis=0) 
    data_mean3 = f3.variables['pH'][:,:,:].mean(axis=0) 
    f1.close()
    f2.close()
    f3.close()
    
    #----
    # extract model data in bottom layer
    #----
    omegaA_1 = np.zeros([len(lon)])
    omegaC_1 = np.zeros([len(lon)])
    pH_1     = np.zeros([len(lon)])
    for i in tqdm(np.arange(1,len(lon)+1)):  
    # double-check if this is correct with python indexing (didn't work without the "-1")
        if lat[i-1]<=50.0: # only process nodes in the SO
            a1 = data_mean1[:,i-1]
            a = a1.mask==False
            b1 = np.where(a==True)[0]
            if len(b1)>0:
                omegaA_1[i-1] = a1[b1[-1]]
            del a1,a,b1
            a1 = data_mean2[:,i-1]
            a = a1.mask==False
            b1 = np.where(a==True)[0]
            if len(b1)>0:
                omegaC_1[i-1] = a1[b1[-1]]
            del a1,a,b1
            a1 = data_mean3[:,i-1]
            a = a1.mask==False
            b1 = np.where(a==True)[0]
            if len(b1)>0:
                pH_1[i-1] = a1[b1[-1]]
            del a1,a,b1

    del data_mean1,data_mean2,data_mean3
        
    omegaA_bottom_model1[yy,:] = omegaA_1 
    omegaC_bottom_model1[yy,:] = omegaC_1 
    pH_bottom_model1[yy,:]     = pH_1 
    del pH_1,omegaA_1,omegaC_1
    
# get decadal avg
omegaA_bottom_model1 = np.nanmean(omegaA_bottom_model1,axis=0)
omegaC_bottom_model1 = np.nanmean(omegaC_bottom_model1,axis=0)
pH_bottom_model1     = np.nanmean(pH_bottom_model1,axis=0)



Process year 1990 of simAssp585


100%|██████████| 72411/72411 [00:02<00:00, 25845.93it/s]


Process year 1991 of simAssp585


100%|██████████| 72411/72411 [00:02<00:00, 25739.30it/s]


Process year 1992 of simAssp585


...

100%|██████████| 72411/72411 [00:02<00:00, 26216.69it/s]


Process year 1998 of simAssp585


100%|██████████| 72411/72411 [00:02<00:00, 26100.61it/s]


Process year 1999 of simAssp585


100%|██████████| 72411/72411 [00:02<00:00, 26190.13it/s]


In [22]:
#-----
# save decadal avg as netcdf file
#-----
    
save_netcdf = True
# only save every year (takes too much time otherwise)
if save_netcdf:
    savepath = '/pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/bottom_fields/'
    
    if year_list1[0]==2090:
        time_str = '2090s'
    elif year_list1[0]==1990:
        time_str = '1990s'
    
    for vv in range(0,len(vari_list)):
        print(vari_list[vv])
        if year_list1[0]==1990:
            netcdf_name = vari_list[vv]+'_'+time_str+'_'+which_sim[:-6]+'.nc'
        else:
            netcdf_name = vari_list[vv]+'_'+time_str+'_'+which_sim+'.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.mocsy_source = 'https://github.com/jamesorr/mocsy'
            w_nc_fid.mocsy_dir    = '/home/ollie/ncara/scripts/mocsy/mocsy'
            w_nc_fid.script    = '/global/homes/c/cnissen/scripts/plot_PAPER_carbonate_chemistry_save_bottom_fields.ipynb'
            # create dimension & variable
            w_nc_fid.createDimension('nodes_2d', len(topo)) 
            w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('nodes_2d'))
            w_nc_fid.close()
            
        else: # file exists, but check if variables exist as well
            w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
            try:
                w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('nodes_2d'))
            except: 
                print (vari_list[vv]+' exists in file')
                #if yy==0:
                #    print (vari_list[vv]+' exists in file')
                #pass
            w_nc_fid.close()
                    
        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
        if vari_list[vv] in ['omegaA_bottom']:
            print('written omegaA')
            w_nc_fid.variables[vari_list[vv]][:] = omegaA_bottom_model1[:]
        elif vari_list[vv] in ['omegaC_bottom']:
            print('written omegaC')
            w_nc_fid.variables[vari_list[vv]][:] = omegaC_bottom_model1[:]
        elif vari_list[vv] in ['pH_bottom']:
            print('written pH')
            w_nc_fid.variables[vari_list[vv]][:] = pH_bottom_model1[:]
        w_nc_fid.close()  
        del netcdf_name
                
print ('done')


omegaA_bottom
Create file /pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/bottom_fields/omegaA_bottom_1990s_simA.nc
written omegaA
omegaC_bottom
Create file /pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/bottom_fields/omegaC_bottom_1990s_simA.nc
written omegaC
pH_bottom
Create file /pscratch/sd/c/cnissen/carbonate_chemistry_3D_v2/bottom_fields/pH_bottom_1990s_simA.nc
written pH
done
